In [1]:
import pandas as pd 
import numpy as np 

# 1: Load Data:

In [5]:
profiles = pd.read_json('../starbuck_capstone/data/profile.json',orient='records',lines=True)
transcripts = pd.read_json('../starbuck_capstone/data/transcript.json',orient='records',lines=True)
promotions = pd.read_json('../starbuck_capstone/data/portfolio.json',orient='records',lines=True)

In [7]:
print('shape of profile:',profiles.shape)
print('shape of transcript:',transcripts.shape)
print('shape of promotions:',promotions.shape)

shape of profile: (17000, 5)
shape of transcript: (306534, 4)
shape of promotions: (10, 6)


## 1.1: Transcript:

In [8]:
transcripts

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714


In [9]:
transcripts.event.value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

__Issues:__
1. Transactions and offers are in single column. 
2. Time seems to bit out of order. 

1.Transactions and offers are in single column. 

In [98]:
def fetch_from_json(json):
    try:
        return json['amount']
    except(KeyError):
        if 'offer id' in json:
            return json['offer id']
        else:
            return json['offer_id']
    


def separate_tran_offer(df):
    cols = ['offer received','offer viewed','offer completed']
    # add offer where col exist # 
    df.loc[df.event.isin(cols),"offer"] = (df.loc[df.event.isin(cols),"value"]).apply(fetch_from_json)
    # add amount where transaction exist #
    df.loc[df.event=='transaction',"amount"] = df.loc[df.event=='transaction',"value"].apply(fetch_from_json)
    return df

separate_tran_offer(transcripts).head(5)

,person,event,value,time,test,offer,amount
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,1.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,1.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,1.0,2906b810c7d4411798c6938adc9daaa5,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,1.0,fafdcd668e3743c1bb461111dcafc2a4,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,1.0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN
